In [40]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [41]:
filepath = '../data/Car Dataset 1945-2020.csv'

# Load data
dt = pd.read_csv(filepath)

print(dt.columns)
print(dt.shape)
dt.head()


Index(['id_trim', 'Make', 'Modle', 'Generation', 'Year_from', 'Year_to',
       'Series', 'Trim', 'Body_type', 'load_height_mm', 'number_of_seats',
       'length_mm', 'width_mm', 'height_mm', 'wheelbase_mm', 'front_track_mm',
       'rear_track_mm', 'curb_weight_kg', 'wheel_size_r14',
       'ground_clearance_mm', 'trailer_load_with_brakes_kg', 'payload_kg',
       'back_track_width_mm', 'front_track_width_mm', 'clearance_mm',
       'full_weight_kg', 'front_rear_axle_load_kg', 'max_trunk_capacity_l',
       'cargo_compartment_length_width_height_mm', 'cargo_volume_m3',
       'minimum_trunk_capacity_l', 'maximum_torque_n_m', 'injection_type',
       'overhead_camshaft', 'cylinder_layout', 'number_of_cylinders',
       'compression_ratio', 'engine_type', 'valves_per_cylinder', 'boost_type',
       'cylinder_bore_mm', 'stroke_cycle_mm', 'engine_placement',
       'cylinder_bore_and_stroke_cycle_mm', 'turnover_of_maximum_torque_rpm',
       'max_power_kw', 'presence_of_intercooler', 'ca

C:\Users\caioa\AppData\Local\Temp\ipykernel_19636\1220355565.py:4: DtypeWarning: Columns (9,11,12,13,14,15,16,17,19,20,24,27,29,33,43,44,47,49,51,69,75,77) have mixed types. Specify dtype option on import or set low_memory=False.
  dt = pd.read_csv(filepath)


,id_trim,Make,Modle,Generation,Year_from,Year_to,Series,Trim,Body_type,load_height_mm,...,front_suspension,steering_type,car_class,country_of_origin,number_of_doors,safety_assessment,rating_name,battery_capacity_KW_per_h,electric_range_km,charging_time_h
0,1,AC,ACE,1 generation,1993.0,2000.0,Cabriolet,3.5 MT,Cabriolet,NaN,...,Helical springs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,AC,ACE,1 generation,1993.0,2000.0,Cabriolet,4.6 MT,Cabriolet,NaN,...,Helical springs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,AC,ACE,1 generation,1993.0,2000.0,Cabriolet,4.9 AT,Cabriolet,NaN,...,Helical springs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,AC,ACE,1 generation,1993.0,2000.0,Roadster,2.9 AT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,AC,ACE,1 generation,1993.0,2000.0,Roadster,2.9 MT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
dt.describe()

,id_trim,Year_from,Year_to,payload_kg,back_track_width_mm,front_track_width_mm,full_weight_kg,minimum_trunk_capacity_l,number_of_cylinders,valves_per_cylinder,...,engine_hp,number_of_gears,turning_circle_m,mixed_fuel_consumption_per_100_km_l,acceleration_0_100_km/h_s,city_fuel_per_100km_l,CO2_emissions_g/km,highway_fuel_per_100km_l,number_of_doors,electric_range_km
count,70823.000000,70586.00000,70189.000000,23799.000000,11198.000000,11204.000000,39682.000000,45953.000000,59544.000000,59334.000000,...,59877.000000,58292.000000,40708.000000,40566.000000,40181.000000,39043.000000,1829.000000,38769.000000,13124.000000,15.000000
mean,35477.818788,1997.06524,1913.441978,605.546199,1477.303179,1482.623527,2067.129127,473.017170,4.972827,3.254138,...,166.659853,4.999074,11.255127,8.675676,10.676648,11.081615,156.235648,6.985210,4.026973,50.800000
std,20494.213522,14.99201,415.392957,320.441908,96.061918,92.701493,619.628212,316.859243,1.585268,0.975289,...,93.269952,1.220068,1.306523,3.854646,3.608049,4.477495,51.822704,2.719996,1.111153,23.170794
min,1.000000,1904.00000,0.000000,145.000000,1050.000000,1105.000000,690.000000,11.000000,1.000000,1.000000,...,5.000000,1.000000,5.100000,0.600000,1.970000,2.100000,13.000000,2.100000,1.000000,30.000000
25%,17724.500000,1988.00000,1994.000000,465.000000,1425.000000,1430.000000,1680.000000,338.000000,4.000000,2.000000,...,105.000000,4.000000,10.500000,6.200000,8.300000,8.000000,120.000000,5.200000,3.000000,34.000000
50%,35453.000000,2000.00000,2005.000000,530.000000,1475.000000,1481.000000,1950.000000,436.000000,4.000000,4.000000,...,141.000000,5.000000,11.000000,7.900000,10.500000,10.300000,146.000000,6.300000,4.000000,46.000000
75%,53240.500000,2008.00000,2013.000000,615.000000,1542.000000,1542.000000,2270.000000,515.000000,6.000000,4.000000,...,203.000000,6.000000,11.800000,9.900000,12.500000,13.100000,178.000000,8.000000,5.000000,52.500000
max,70987.000000,2020.00000,2020.000000,3334.000000,1869.000000,1869.000000,5352.000000,4440.000000,16.000000,6.000000,...,1914.000000,10.000000,71.000000,36.500000,50.000000,43.100000,547.000000,28.000000,5.000000,106.000000


In [43]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics.pairwise import cosine_similarity

dt['Model']=dt['Modle'].str.lower()
dt.drop('Modle', axis=1, inplace=True)


numerical_features = [
    'Year_from', 'number_of_seats', 'curb_weight_kg', 
    'engine_hp', 'number_of_gears', 'acceleration_0_100_km/h_s',
    'mixed_fuel_consumption_per_100_km_l'
]

categorical_features = [
    'Body_type', 'engine_type', 'drive_wheels', 'transmission', 'country_of_origin'
]

# Create a new DataFrame with only the selected features
df_rec = dt[numerical_features + categorical_features].copy()


# Fill missing numerical values with the median
for col in numerical_features:
    df_rec[col] = pd.to_numeric(df_rec[col], errors='coerce')
    median_val = df_rec[col].median()
    df_rec[col].fillna(median_val, inplace=True)

# Fill missing categorical values with a placeholder
for col in categorical_features:
    df_rec[col].fillna('Unknown', inplace=True)


preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'
)


feature_matrix = preprocessor.fit_transform(df_rec)

print("Preprocessing complete.")
print("Shape of the final feature matrix:", feature_matrix.shape)


C:\Users\caioa\AppData\Local\Temp\ipykernel_19636\2859679855.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_rec[col].fillna(median_val, inplace=True)
C:\Users\caioa\AppData\Local\Temp\ipykernel_19636\2859679855.py:31: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For examp

Preprocessing complete.
Shape of the final feature matrix: (70823, 73)


In [44]:
from sklearn.metrics.pairwise import cosine_similarity
dt = dt.reset_index()
indices = pd.Series(dt.index, index=dt['Model'])

def get_recommendations(model_name, feature_matrix=feature_matrix, dataframe=dt, indices_map=indices):
    # Normalize model names in the indices map for case-insensitive and trimmed matching
    normalized_indices = pd.Series(indices_map.values, index=indices_map.index.str.strip().str.lower())

    # Normalize the input model name
    model_name_norm = model_name.strip().lower()

    try:
        # Get all indices of the car that matches the normalized model name.
        idx_list = normalized_indices[model_name_norm]
        # If multiple, pick the first one for similarity calculation
        if isinstance(idx_list, (pd.Series, np.ndarray, list)):
            idx = idx_list.iloc[0] if hasattr(idx_list, "iloc") else idx_list[0]
        else:
            idx = idx_list
    except KeyError:
        return f"Model '{model_name}' not found in the dataset."

    # Get the feature vector for the specific model
    model_vector = feature_matrix[idx]

    # Calculate the cosine similarity of our car against ALL other cars.
    sim_scores = cosine_similarity(model_vector, feature_matrix)

    # Get the similarity scores as a list of (index, score) tuples
    sim_scores = list(enumerate(sim_scores[0]))

    # Sort the cars based on the similarity scores in descending order
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar cars, excluding the car itself (which is at index 0)
    sim_scores = sim_scores[1:]

    # Get the original indices of the top similar cars, avoiding duplicates by 'Make' and 'Model'
    seen = set()
    recommendations = []
    for i, score in sim_scores:
        make = dataframe.iloc[i]['Make']
        model = dataframe.iloc[i]['Model']
        key = (make, model)
        if key not in seen:
            seen.add(key)
            recommendations.append(i)
        if len(recommendations) == 10:
            break

    # Return the top 10 most similar cars
    return dataframe[['Make', 'Model', 'Year_from', 'Body_type', 'engine_hp', 'number_of_seats', 'number_of_doors']].iloc[recommendations]

# --- Test the new function ---
print("\nCars similar to 'hb20':")
display(get_recommendations('hb20'))

print("\nCars similar to 'Camry':")
display(get_recommendations('Camry'))


Cars similar to 'hb20':


,Make,Model,Year_from,Body_type,engine_hp,number_of_seats,number_of_doors
27582,Hyundai,hb20,2012.0,Hatchback,122.0,5,NaN
66663,Volkswagen,fox,2009.0,Hatchback,72.0,5,NaN
43950,Nissan,micra,2007.0,Hatchback,65.0,5,NaN
66719,Volkswagen,gol,2012.0,Hatchback,72.0,5,NaN
22323,Ford,fiesta,2008.0,Hatchback,72.0,5,NaN
70764,ZAZ,sens,2007.0,Hatchback,70.0,5,NaN
20131,FAW,vita,2007.0,Hatchback,92.0,5,NaN
32320,Lifan,smily,2008.0,Hatchback,88.0,5,NaN
51336,Ravon,matiz,2015.0,Hatchback,51.0,5,NaN
51952,Renault,clio,2009.0,Hatchback,60.0,5,NaN



Cars similar to 'Camry':


,Make,Model,Year_from,Body_type,engine_hp,number_of_seats,number_of_doors
25405,Honda,ballade,1980.0,NaN,NaN,5,4.0
29157,Isuzu,kb,1980.0,NaN,NaN,5,4.0
32877,Mazda,323,1980.0,NaN,NaN,5,3.0
40143,Mitsubishi,galant,1980.0,NaN,NaN,5,5.0
44255,Nissan,patrol,1980.0,NaN,NaN,5,3.0
59288,Toyota,camry,1980.0,NaN,NaN,5,4.0
59805,Toyota,chaser,1980.0,NaN,NaN,5,4.0
60969,Toyota,cresta,1980.0,NaN,NaN,5,4.0
62009,Toyota,land cruiser,1980.0,NaN,NaN,5,5.0
18479,Daihatsu,charmant,1981.0,NaN,NaN,5,4.0


In [45]:
# --- Feature Engineering ---
# Ensure columns are numeric before division, coercing errors to NaN
df_rec['engine_hp'] = pd.to_numeric(df_rec['engine_hp'], errors='coerce')
df_rec['curb_weight_kg'] = pd.to_numeric(df_rec['curb_weight_kg'], errors='coerce')

# Calculate Power-to-Weight Ratio (HP per KG)
# We fill any resulting NaNs from the division with the median of the new column
df_rec['power_to_weight_ratio'] = df_rec['engine_hp'] / df_rec['curb_weight_kg']
df_rec['power_to_weight_ratio'].fillna(df_rec['power_to_weight_ratio'].median(), inplace=True)

numerical_features = [
    'Year_from', 'number_of_seats', 'curb_weight_kg', 
    'engine_hp', 'number_of_gears', 'acceleration_0_100_km/h_s',
    'mixed_fuel_consumption_per_100_km_l',
    'power_to_weight_ratio', 'number_of_doors'
]


# Separate features by importance
high_importance_num = ['Year_from', 'engine_hp', 'mixed_fuel_consumption_per_100_km_l']
high_importance_cat = ['Body_type', 'engine_type']

medium_importance_num = ['number_of_seats', 'curb_weight_kg', 'number_of_gears', 'acceleration_0_100_km/h_s']
medium_importance_cat = ['drive_wheels', 'transmission', 'country_of_origin']

# Create a preprocessor that applies different weights
# We do this by chaining a scaler with a custom weighting function (via FunctionTransformer)
from sklearn.preprocessing import FunctionTransformer

# Define weighting functions
def apply_high_weight(x):
    return x * 2.0  # Double the importance

def apply_medium_weight(x):
    return x * 1.0 # Keep importance as is

preprocessor = ColumnTransformer(
    transformers=[
        # High importance features
        ('num_high', MinMaxScaler(), high_importance_num),
        ('cat_high', OneHotEncoder(handle_unknown='ignore'), high_importance_cat),
        
        # Medium importance features
        ('num_medium', MinMaxScaler(), medium_importance_num),
        ('cat_medium', OneHotEncoder(handle_unknown='ignore'), medium_importance_cat)
    ],
    remainder='passthrough'
)

feature_matrix = preprocessor.fit_transform(df_rec)

print("Preprocessing V2 complete.")
print("Shape of the final feature matrix:", feature_matrix.shape)


C:\Users\caioa\AppData\Local\Temp\ipykernel_19636\4261258432.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_rec['power_to_weight_ratio'].fillna(df_rec['power_to_weight_ratio'].median(), inplace=True)


Preprocessing V2 complete.
Shape of the final feature matrix: (70823, 74)


In [46]:
# Finds the best matching cars based on a dictionary of user preferences.
def get_recommendations_from_prefs(prefs, preprocessor, feature_matrix, dataframe, top_n=10):
    # Create a DataFrame with a single row representing the user's ideal car
    ideal_car = pd.DataFrame([prefs])

    # Ensure all required columns are present and in the same order as df_rec
    for col in df_rec.columns:
        if col not in ideal_car.columns:
            ideal_car[col] = np.nan
    ideal_car = ideal_car[df_rec.columns]

    # Fill missing values as in df_rec
    for col in df_rec.columns:
        if col in df_rec.select_dtypes(include=[np.number]).columns:
            median_val = df_rec[col].median()
            ideal_car[col] = pd.to_numeric(ideal_car[col], errors='coerce').fillna(median_val)
        else:
            ideal_car[col] = ideal_car[col].fillna('Unknown')

    # Use the *already fitted* preprocessor to transform the ideal car into a feature vector
    ideal_vector = preprocessor.transform(ideal_car)

    # Calculate similarity and find top matches (same logic as before)
    sim_scores = cosine_similarity(ideal_vector, feature_matrix)
    sim_scores = list(enumerate(sim_scores[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the top N car indices
    car_indices = [i[0] for i in sim_scores[:top_n]]

    return dataframe[['Make', 'Model', 'Year_from', 'Body_type', 'engine_hp']].iloc[car_indices]

print("\n--- Recommendations based on User Preferences ---")

feature_matrix = preprocessor.fit_transform(df_rec)

user_preferences = {
    'Body_type': 'Sedan',
    'engine_hp': 250,
    'Year_from': 2018,
    'transmission': 'Automatic',
    'mixed_fuel_consumption_per_100_km_l': 8.0
}

display(get_recommendations_from_prefs(user_preferences, preprocessor, feature_matrix, dt))



--- Recommendations based on User Preferences ---


,Make,Model,Year_from,Body_type,engine_hp
69987,Volvo,s60,2018.0,Sedan,328.0
69986,Volvo,s60,2018.0,Sedan,318.0
24502,Genesis,g70,2017.0,Sedan,370.0
30744,Kia,k900,2018.0,Sedan,413.0
30742,Kia,k900,2018.0,Sedan,249.0
30743,Kia,k900,2018.0,Sedan,249.0
24508,Genesis,g80,2016.0,Sedan,315.0
24507,Genesis,g80,2016.0,Sedan,315.0
24505,Genesis,g80,2016.0,Sedan,370.0
24498,Genesis,g70,2017.0,Sedan,252.0


In [47]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD

# Define the numerical and categorical features
numerical_features = [
    'Year_from', 'number_of_seats', 'curb_weight_kg', 
    'engine_hp', 'number_of_gears', 'acceleration_0_100_km/h_s',
    'mixed_fuel_consumption_per_100_km_l',
    'power_to_weight_ratio'
]
categorical_features = [
    'Body_type', 'engine_type', 'drive_wheels', 'transmission', 'country_of_origin'
]

# Create a pipeline for categorical features: OneHotEncode then reduce dimension
categorical_pipeline = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore')),
    # Reduce the many encoded columns down to 50 "latent feature" components
    ('svd', TruncatedSVD(n_components=50, random_state=42)) 
])

# Create the master preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_features),
        # Apply the entire pipeline to the categorical features
        ('cat', categorical_pipeline, categorical_features)
    ],
    remainder='passthrough'
)

#Now the output will be more compact and potentially capture more meaningful patterns.
feature_matrix = preprocessor.fit_transform(df_rec)

user_preferences = {
    'Body_type': 'Sedan',
    'engine_hp': 250,
    'Year_from': 2018,
    'transmission': 'Automatic',
    'mixed_fuel_consumption_per_100_km_l': 8.0
}

print("\n--- Recommendations based on User Preferences ---")
display(get_recommendations_from_prefs(user_preferences, preprocessor, feature_matrix, dt))


print("\nCars similar to 'i20':")
display(get_recommendations('celta'))


--- Recommendations based on User Preferences ---


,Make,Model,Year_from,Body_type,engine_hp
69987,Volvo,s60,2018.0,Sedan,328.0
69986,Volvo,s60,2018.0,Sedan,318.0
24502,Genesis,g70,2017.0,Sedan,370.0
30742,Kia,k900,2018.0,Sedan,249.0
30744,Kia,k900,2018.0,Sedan,413.0
30743,Kia,k900,2018.0,Sedan,249.0
24508,Genesis,g80,2016.0,Sedan,315.0
24507,Genesis,g80,2016.0,Sedan,315.0
24505,Genesis,g80,2016.0,Sedan,370.0
24498,Genesis,g70,2017.0,Sedan,252.0



Cars similar to 'i20':


,Make,Model,Year_from,Body_type,engine_hp,number_of_seats,number_of_doors
9350,Chevrolet,celta,2000.0,Hatchback,70.0,5,NaN
26949,Hyundai,amica,1999.0,Hatchback,56.0,5,NaN
26968,Hyundai,atos,1999.0,Hatchback,56.0,5,NaN
48928,Peugeot,206,1998.0,Hatchback,60.0,5,NaN
43764,Nissan,march,1999.0,Hatchback,60.0,5,NaN
22277,Ford,fiesta,1999.0,Hatchback,60.0,5,NaN
51795,Renault,clio,1998.0,Hatchback,60.0,5,NaN
26384,Honda,logo,2000.0,Hatchback,66.0,5,NaN
47005,Opel,corsa,1997.0,Hatchback,60.0,5,NaN
43917,Nissan,micra,1999.0,Hatchback,60.0,5,NaN


In [48]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import RandomForestClassifier # Import the model

In [49]:
#Implementation of RandomForest based recommendation system
# Create a combined 'car_name' column for easy lookup
df = pd.read_csv(filepath)
df['Model']=df['Modle'].str.lower()
df.drop('Modle', axis=1, inplace=True)
df_rec = df.dropna(subset=['Make', 'Model', 'Body_type'])

# Clean up number_of_seats column to ensure it's numeric
df_rec['number_of_seats'] = pd.to_numeric(df_rec['number_of_seats'], errors='coerce')

# Ensure columns are numeric before division, coercing errors to NaN
df_rec['engine_hp'] = pd.to_numeric(df_rec['engine_hp'], errors='coerce')
df_rec['curb_weight_kg'] = pd.to_numeric(df_rec['curb_weight_kg'], errors='coerce')

# Calculate Power-to-Weight Ratio (HP per KG)
# We fill any resulting NaNs from the division with the median of the new column
df_rec['power_to_weight_ratio'] = df_rec['engine_hp'] / df_rec['curb_weight_kg']
df_rec['power_to_weight_ratio'].fillna(df_rec['power_to_weight_ratio'].median(), inplace=True)

# Define the numerical and categorical features
numerical_features = [
    'Year_from', 'number_of_seats', 'curb_weight_kg', 
    'engine_hp', 'number_of_gears', 'acceleration_0_100_km/h_s',
    'mixed_fuel_consumption_per_100_km_l',
    'power_to_weight_ratio'
]
categorical_features = [
    'Body_type', 'engine_type', 'drive_wheels', 'transmission', 'country_of_origin'
]

# Create preprocessing pipelines
# Using median for numerical imputation is often more robust to outliers
numerical_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='median')),
])

categorical_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('one-hot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Create the master preprocessor
preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_pipeline, numerical_features),
    ('cat', categorical_pipeline, categorical_features)
], remainder='drop')

# Create the feature matrix
feature_matrix = preprocessor.fit_transform(df_rec)

C:\Users\caioa\AppData\Local\Temp\ipykernel_19636\2649976146.py:3: DtypeWarning: Columns (9,11,12,13,14,15,16,17,19,20,24,27,29,33,43,44,47,49,51,69,75,77) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
C:\Users\caioa\AppData\Local\Temp\ipykernel_19636\2649976146.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rec['number_of_seats'] = pd.to_numeric(df_rec['number_of_seats'], errors='coerce')
C:\Users\caioa\AppData\Local\Temp\ipykernel_19636\2649976146.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

In [50]:
def get_recommendations_tree(car_name, preprocessor, feature_matrix, df, top_n=10):
    try:
        # Find the index of the input car
        car_index = df[df['Model'].str.lower() == car_name.lower()].index[0]
    except IndexError:
        return f"Car '{car_name}' not found in the dataset."

    # --- Create the Target Variable ---
    # We create a target array where the selected car is '1' and all others are '0'
    y = np.zeros(feature_matrix.shape[0])
    y[car_index] = 1

    # --- Train the Model ---
    # We train a classifier to distinguish the target car from the others.
    # 'class_weight="balanced"' is crucial because we have a huge imbalance (one '1' vs. many '0's)
    model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight="balanced", n_jobs=-1)
    model.fit(feature_matrix, y)

    # --- Get Similarity Scores ---
    # The model's prediction probability for class '1' serves as our similarity score.
    probabilities = model.predict_proba(feature_matrix)[:, 1]

    # --- Find Top Recommendations ---
    # We sort the probabilities and get the indices of the top N cars.
    # We exclude the input car itself from the recommendations.
    similar_car_indices = probabilities.argsort()[-top_n-1:-1]

    # Return the recommended cars in descending order of similarity
    recommended_cars = df.iloc[similar_car_indices][::-1]

    # Add similarity score to the output
    recommended_cars['similarity_score'] = probabilities[similar_car_indices][::-1]

    return recommended_cars[[ 'Make', 'Model', 'Body_type', 'engine_hp', 'Year_from', 'similarity_score']]

In [51]:
# Reset index so that feature_matrix and df_rec are aligned
df_rec_reset = df_rec.reset_index(drop=True)

print("\n--- Tree-Based Recommendations for 'Corsa' ---")
display(get_recommendations_tree('Corsa', preprocessor, feature_matrix, df_rec_reset))


--- Tree-Based Recommendations for 'Corsa' ---


,Make,Model,Body_type,engine_hp,Year_from,similarity_score
8285,Chevrolet,corsa,Sedan,64.0,1994.0,0.06
7116,Chevrolet,celta,Hatchback,77.0,2011.0,0.04
46492,Toyota,prius,Sedan,58.0,1997.0,0.03
20606,Honda,insight,Hatchback,77.0,1999.0,0.02
45053,Toyota,corsa,Sedan,67.0,1994.0,0.01
49624,Volkswagen,lupo,Hatchback,61.0,1998.0,0.01
20605,Honda,insight,Hatchback,82.0,1999.0,0.01
13063,Chevrolet,spark,Hatchback,64.0,2003.0,0.01
7114,Chevrolet,celta,Hatchback,70.0,2000.0,0.01
827,Audi,a2,Hatchback,61.0,1999.0,0.01


In [ ]:
# Run this cell to import everything you need
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display, clear_output

# Assume all your previous code for data loading, preprocessing, and the
# get_recommendations_tree() function is already in memory.

def show_similar_cars(car_name):
    #####  Wrapper function to display recommendations.
    # Call your tree-based recommendation function
    recommendations = get_recommendations(car_name)
    display(recommendations)

# Use interact to instantly create a UI
interact(show_similar_cars, car_name=df_rec['Model'].unique().tolist());

interactive(children=(Dropdown(description='car_name', options=('ace', 'aceca', 'cobra', 'cl', 'csx', 'el', 'i…